In [3]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/new_master_dataset.csv")
df

Mounted at /content/drive


mag                                               path  \
0     100  /content/drive/MyDrive/Breast Cancer Project/b...   
1     100  /content/drive/MyDrive/Breast Cancer Project/b...   
2     100  /content/drive/MyDrive/Breast Cancer Project/b...   
3     100  /content/drive/MyDrive/Breast Cancer Project/b...   
4     100  /content/drive/MyDrive/Breast Cancer Project/b...   
...   ...                                                ...   
7904  400  /content/drive/MyDrive/Breast Cancer Project/b...   
7905  400  /content/drive/MyDrive/Breast Cancer Project/b...   
7906  400  /content/drive/MyDrive/Breast Cancer Project/b...   
7907  400  /content/drive/MyDrive/Breast Cancer Project/b...   
7908  400  /content/drive/MyDrive/Breast Cancer Project/b...   

                            filename      class slide_id tumor_type  
0     SOB_B_A-14-22549CD-100-001.png     benign  22549CD          A  
1     SOB_B_A-14-22549CD-100-002.png     benign  22549CD          A  
2     SOB_B_A-14-22549CD-100-003.png     benign  22549CD          A  
3     SOB_B_A-14-22549CD-100-004.png     benign  22549CD          A  
4     SOB_B_A-14-22549CD-100-005.png     benign  22549CD          A  
...                              ...        ...      ...        ...  
7904   SOB_M_PC-14-15704-400-031.png  malignant    15704         PC  
7905   SOB_M_PC-14-15704-400-032.png  malignant    15704         PC  
7906   SOB_M_PC-14-15704-400-033.png  malignant    15704         PC  
7907   SOB_M_PC-14-15704-400-034.png  malignant    15704         PC  
7908   SOB_M_PC-14-15704-400-035.png  malignant    15704         PC  

[7909 rows x 6 columns]

In [4]:
groupd_df = df.groupby("tumor_type")

# You can then perform various operations on the grouped data, such as getting the count of each tumor type
tumor_type_counts = groupd_df.size()
tumor_type_counts

tumor_type
A      444
DC    3451
F     1014
LC     626
MC     792
PC     560
PT     453
TA     569
dtype: int64

In [5]:
# df_train_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/train_df_100.csv")
# df_test_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/test_df_100.csv")
# df_val_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/val_df_100.csv")

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import os
import torch
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define constants
data_dirs = ["TA", "PT", "PC", "MC", "LC", "F", "DC", "A"]
data_root = "/content/drive/MyDrive/Breast Cancer Project/IW/200"  # Replace with the root directory of your data
train_split = 0.7

# Create a list to store the paths and labels of all images
all_data = []

# Populate the list with paths and labels
for label, folder in enumerate(data_dirs):
    folder_path = os.path.join(data_root, folder)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        all_data.append((image_path, label))

# Split data into training and testing sets
train_data, test_data = train_test_split(all_data, train_size=train_split, shuffle=True, random_state=42)

# Define custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')  # Open image and convert to RGB mode
        if self.transform:
            img = self.transform(img)
        label_tensor = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        return img, label_tensor

# Image preprocessing with augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((528, 528)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

# Image preprocessing without augmentation for testing and validation
test_val_transform = transforms.Compose([
    transforms.Resize((528, 528)),
    transforms.ToTensor()
])

# Create custom datasets
train_dataset = CustomDataset(train_data, transform=train_transform)
test_dataset = CustomDataset(test_data, transform=test_val_transform)

# DataLoaders for batching and shuffling
batch_size = 6 # Define the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load model directly
processor = AutoImageProcessor.from_pretrained("google/efficientnet-b6")
model = AutoModelForImageClassification.from_pretrained("google/efficientnet-b6").to(device)

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/70.2k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/173M [00:00<?, ?B/s]

...

Epoch 11/15, Train Loss: 0.2188, Train Accuracy: 92.05%, Val Loss: 0.5809, Val Accuracy: 82.12%


Epoch 12/15, Train Loss: 0.1937, Train Accuracy: 92.83%, Val Loss: 0.6339, Val Accuracy: 82.62%
Epoch 00012: reducing learning rate of group 0 to 1.0000e-05.


Epoch 13/15, Train Loss: 0.1395, Train Accuracy: 95.24%, Val Loss: 0.4771, Val Accuracy: 85.60%


Epoch 14/15, Train Loss: 0.1332, Train Accuracy: 95.10%, Val Loss: 0.3494, Val Accuracy: 87.09%


Epoch 15/15, Train Loss: 0.1186, Train Accuracy: 95.95%, Val Loss: 0.5558, Val Accuracy: 84.93%
Test Accuracy: 84.93%


In [7]:
from sklearn.metrics import classification_report

# Test the model
model.eval()
test_correct = 0
test_total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print classification report
print("Classification Report:")
report = classification_report(all_labels, all_predicted, digits=3)
print(report)

Test Accuracy: 84.93%
Classification Report:
              precision    recall  f1-score   support

           0      0.786     0.917     0.846        36
           1      0.852     0.742     0.793        31
           2      0.918     0.882     0.900        51
           3      1.000     0.615     0.762        65
           4      0.857     0.250     0.387        48
           5      0.784     0.938     0.854        81
           6      0.833     0.977     0.899       260
           7      1.000     0.938     0.968        32

    accuracy                          0.849       604
   macro avg      0.879     0.782     0.801       604
weighted avg      0.860     0.849     0.833       604



In [8]:
# Train the model
num_epochs = 30
for epoch in range(16, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 17/30, Train Loss: 0.1041, Train Accuracy: 96.10%, Val Loss: 0.6895, Val Accuracy: 79.30%


Epoch 18/30, Train Loss: 0.1003, Train Accuracy: 96.95%, Val Loss: 0.3155, Val Accuracy: 87.75%


Epoch 19/30, Train Loss: 0.0902, Train Accuracy: 97.09%, Val Loss: 0.3911, Val Accuracy: 87.91%


Epoch 20/30, Train Loss: 0.0951, Train Accuracy: 96.24%, Val Loss: 0.4338, Val Accuracy: 86.09%


Epoch 21/30, Train Loss: 0.1008, Train Accuracy: 96.31%, Val Loss: 0.4412, Val Accuracy: 85.60%


...

Epoch 26/30, Train Loss: 0.0690, Train Accuracy: 97.30%, Val Loss: 0.3637, Val Accuracy: 88.74%
Epoch 00025: reducing learning rate of group 0 to 1.0000e-07.


Epoch 27/30, Train Loss: 0.0738, Train Accuracy: 97.44%, Val Loss: 0.4854, Val Accuracy: 86.59%


Epoch 28/30, Train Loss: 0.0740, Train Accuracy: 97.23%, Val Loss: 0.6167, Val Accuracy: 84.11%


Epoch 29/30, Train Loss: 0.0919, Train Accuracy: 97.02%, Val Loss: 0.5008, Val Accuracy: 86.09%


Epoch 30/30, Train Loss: 0.0719, Train Accuracy: 97.59%, Val Loss: 0.4066, Val Accuracy: 88.08%
Epoch 00029: reducing learning rate of group 0 to 1.0000e-08.
Test Accuracy: 88.08%


In [9]:
from sklearn.metrics import classification_report

# Test the model
model.eval()
test_correct = 0
test_total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print classification report
print("Classification Report:")
report = classification_report(all_labels, all_predicted, digits=3)
print(report)

Test Accuracy: 88.08%
Classification Report:
              precision    recall  f1-score   support

           0      0.825     0.917     0.868        36
           1      0.711     0.871     0.783        31
           2      0.940     0.922     0.931        51
           3      0.958     0.708     0.814        65
           4      0.711     0.667     0.688        48
           5      0.851     0.914     0.881        81
           6      0.917     0.938     0.928       260
           7      0.967     0.906     0.935        32

    accuracy                          0.881       604
   macro avg      0.860     0.855     0.854       604
weighted avg      0.885     0.881     0.880       604

